In [62]:
import json
import sys
from sklearn.model_selection import train_test_split
import salty
sys.path.insert(0, '../')

from scripts import *

def vae_to_qspr_trainer(properties,
                        T = [297, 316],
                        P = [99, 102],
                        weights_file=False,
                        save_as=False,
                        smile_max_length = 62,
                        random_state=42,
                        training_epochs=1):
    
    f = open("../data/salt_char_to_index.json","r")
    char_to_index = json.loads(f.read())
    char_set = set(char_to_index.keys())
    char_list = list(char_to_index.keys())
    chars_in_dict = len(char_list)

    
    chemvae = MoleculeVAE()
    if weights_file:
        chemvae.create(char_set, max_length=smile_max_length, weights_file=weights_file, qspr=True)
    else:
        chemvae.create(char_set, max_length=smile_max_length, qspr=True)

    props = properties
    devmodel = salty.aggregate_data(props, T=T, P=P, merge='Union')
    devmodel.Data['smiles_string'] = devmodel.Data['smiles-cation'] + "." + devmodel.Data['smiles-anion']
    salts = devmodel.Data['smiles_string']
    salts = salts.reset_index(drop=True)
    
    property_index = 6
    if prop == 'melting_point':
        property_index = 8
    print(devmodel.Data.columns[-property_index])
    print("total salts in training data:\t{}".format(salts.shape[0]))

    cations = [i.split('.')[0] for i in salts]
    anions = [i.split('.')[1] for i in salts]
    
    print("first training sample:\t{}\t{}".format(cations[0], anions[0]))
    print("last training sample:\t{}\t{}".format(cations[-1], anions[-1]))
    
    padded_cations =  [pad_smiles(i, smile_max_length) for i in cations if pad_smiles(i, smile_max_length)]
    padded_anions =  [pad_smiles(i, smile_max_length) for i in anions if pad_smiles(i, smile_max_length)]
    X_train_cat = np.zeros((len(cations), smile_max_length, chars_in_dict), dtype=np.float32)
    X_train_ani = np.zeros((len(anions), smile_max_length, chars_in_dict), dtype=np.float32)
    Y_train = np.array(devmodel.Data[devmodel.Data.columns[-property_index]])
    
    for i, smile in enumerate(padded_cations):
        for j, char in enumerate(smile):
            X_train_cat[i, j, char_to_index[char]] = 1
    
    for i, smile in enumerate(padded_anions):
        for j, char in enumerate(smile):
            X_train_ani[i, j, char_to_index[char]] = 1

    X_train_cat, X_test_cat, X_train_ani, X_test_ani, Y_train, Y_test = train_test_split(
        X_train_cat, X_train_ani, Y_train, test_size=0.1, random_state=random_state) 

#     history = chemvae.autoencoder.fit(X_train, [X_train, Y_train],
#                             shuffle = False,
#                             validation_data=(X_test, [X_test, Y_test]),
#                             epochs=training_epochs)
#     if save_as:
#         chemvae.save('{}.h5'.format(save_as))
#         with open('history_{}.json'.format(save_as), 'w') as f:
#             json.dump(history.history, f)
    return X_train_cat, X_test_cat, X_train_ani, X_test_ani, Y_train, Y_test

In [63]:
properties = ['density', 'viscosity', 'cpt', 'melting_point',
            'thermal_conductivity']

In [64]:
for prop in properties:
    x_train_cat, x_test_cat, x_train_ani, x_test_ani, y_train, y_test = vae_to_qspr_trainer([prop])
    np.save('{}_x_train_cat'.format(prop), x_train_cat)
    np.save('{}_x_train_ani'.format(prop), x_train_ani)
    np.save('{}_y_train'.format(prop), y_train)
    np.save('{}_x_test_cat'.format(prop), x_test_cat)
    np.save('{}_x_test_ani'.format(prop), x_test_ani)
    np.save('{}_y_test'.format(prop), y_test)
    print('')

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C	[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C	C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F

Viscosity, Pa&#8226;s
total salts in training data:	3277
first training sample:	CCCC[n+]1ccc(cc1)C	[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C	C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F

Heat capacity at constant pressure, J/K/mol
total salts in training data:	1739
first training sample:	CCCC[n+]1ccc(cc1)C	[B-](F)(F)(F)F
last training sample:	CCCCn1cc[n+](c1)C	COCCOCCOS(=O)(=O)[O-]

Normal melting temperature, K
total salts in training data:	389
first training sample:	CCCC[n+]1ccc(cc1)C	[B-](F)(F)(F)F
last training sample:	CN(C)C(=[NH2+])N(C)C	C(=O)(C(F)(F)F)[O-]

Thermal conductivity, W/m/K
total salts in training data:	148
first training sample:	CCCCCCCCn1cc[n+](c1)C	[B-](F)(F)(F)F
last training sample:	CCn1cc[n+](c1)C	COP(=O)([O-])



In [69]:
import numpy as np
from numpy import array
from numpy import argmax
from random import shuffle
import pandas as pd
import random

#Keras build
import keras
from keras import objectives
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Multiply, Add
from keras.optimizers import Adam, Nadam
from keras import backend as K
from keras.objectives import binary_crossentropy #objs or losses
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Layer
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#cation data
cations = pd.read_csv('../data/cations.csv')
cations = cations['smiles_string']
salts = pd.read_csv('../data/salts.csv')
salts = salts['smiles_string']
anions = pd.read_csv('../data/anions.csv')
anions = anions['smiles']

#supporting functions
import sys
sys.path.insert(0, '../')
from scripts import *


#setup model
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

chemvae = MoleculeVAE()
chemvae.create(char_set, 
               mol_inputs=2)
data_size = 100000
histories = []
gen1vae = MoleculeVAE()
gen1vae.create(char_set, qspr=True, mol_inputs=2, weights_file='../models/gen1_2mol_1mil_GDB17_mix_pure_5.h5')

gen2vae = TwoMoleculeOneLatentVAE()
gen2vae.create(char_set, qspr=True, weights_file='../models/gen2_2mol_1mil_GDB17_mix_pure_5.h5')

gen3vae = TwoMoleculeVAE()
gen3vae.create(char_set, char_set, qspr=True, weights_file='../models/gen3_2mol_1mil_GDB17_mix_pure_5.h5')

properties = ['density', 'viscosity', 'cpt', 'melting_point',
            'thermal_conductivity']

In [76]:
epochs = 1
for prop in ['thermal_conductivity']:
    x_train_cat = np.load('../data/{}_x_train_cat.npy'.format(prop))
    x_train_ani = np.load('../data/{}_x_train_ani.npy'.format(prop))
    x_test_cat = np.load('../data/{}_x_test_cat.npy'.format(prop))
    x_test_ani = np.load('../data/{}_x_test_ani.npy'.format(prop))
    y_train = np.load('../data/{}_y_train.npy'.format(prop))
    y_test = np.load('../data/{}_y_test.npy'.format(prop))
    
#     history = gen1vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen1vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen1vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen2vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen2vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen2vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
    history = gen3vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
                          shuffle=False,
                          validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
                          epochs=epochs)
    chemvae.save('../models/gen3vae_{}_{}.h5'.format(prop,epochs))
    with open('../models/history_gen3vae_{}_{}.json'.format(prop,epochs), 'w') as f:
        json.dump(history.history, f)

# epochs = 10
# for prop in properties[:1]:
#     x_train_cat = np.load('{}_x_train_cat.npy'.format(prop))
#     x_train_ani = np.load('{}_x_train_ani.npy'.format(prop))
#     x_test_cat = np.load('{}_x_test_cat.npy'.format(prop))
#     x_test_ani = np.load('{}_x_test_ani.npy'.format(prop))
#     y_train = np.load('{}_y_train.npy'.format(prop))
#     y_test = np.load('{}_y_test.npy'.format(prop))
    
#     history = gen1vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen1vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen1vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen2vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen2vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen2vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen3vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen3vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen3vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)

        
# epochs = 30
# for prop in properties[:1]:
#     x_train_cat = np.load('{}_x_train_cat.npy'.format(prop))
#     x_train_ani = np.load('{}_x_train_ani.npy'.format(prop))
#     x_test_cat = np.load('{}_x_test_cat.npy'.format(prop))
#     x_test_ani = np.load('{}_x_test_ani.npy'.format(prop))
#     y_train = np.load('{}_y_train.npy'.format(prop))
#     y_test = np.load('{}_y_test.npy'.format(prop))
    
#     history = gen1vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen1vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen1vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen2vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen2vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen2vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen3vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen3vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen3vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
    
# epochs = 100
# for prop in properties[:1]:
#     x_train_cat = np.load('{}_x_train_cat.npy'.format(prop))
#     x_train_ani = np.load('{}_x_train_ani.npy'.format(prop))
#     x_test_cat = np.load('{}_x_test_cat.npy'.format(prop))
#     x_test_ani = np.load('{}_x_test_ani.npy'.format(prop))
#     y_train = np.load('{}_y_train.npy'.format(prop))
#     y_test = np.load('{}_y_test.npy'.format(prop))
    
#     history = gen1vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen1vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen1vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen2vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen2vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen2vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen3vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     chemvae.save('gen3vae_{}_{}.h5'.format(prop,epochs))
#     with open('history_gen3vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)

Train on 133 samples, validate on 15 samples
Epoch 1/1
133/133 [==============================] - 4s 33ms/step - loss: 6.5298 - cation_decoded_mean_loss: 0.1657 - anion_decoded_mean_loss: 0.2104 - qspr_loss: 6.1538 - cation_decoded_mean_acc: 0.9886 - cation_decoded_mean_mean_squared_error: 5.1617e-04 - anion_decoded_mean_acc: 0.9603 - anion_decoded_mean_mean_squared_error: 8.7132e-04 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 6.1538 - val_loss: 4.4551 - val_cation_decoded_mean_loss: 0.1587 - val_anion_decoded_mean_loss: 0.2812 - val_qspr_loss: 4.0152 - val_cation_decoded_mean_acc: 0.9860 - val_cation_decoded_mean_mean_squared_error: 6.1821e-04 - val_anion_decoded_mean_acc: 0.9290 - val_anion_decoded_mean_mean_squared_error: 0.0013 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 4.0152


In [77]:
# epochs = 1
# for prop in properties:
#     gen1vae = MoleculeVAE()
#     gen1vae.create(char_set, qspr=True, mol_inputs=2, weights_file='../models/gen1_2mol_1mil_GDB17_mix_pure_5.h5')
#     gen2vae = TwoMoleculeOneLatentVAE()
#     gen2vae.create(char_set, qspr=True, weights_file='../models/gen2_2mol_1mil_GDB17_mix_pure_5.h5')
#     gen3vae = TwoMoleculeVAE()
#     gen3vae.create(char_set, char_set, qspr=True, weights_file='../models/gen3_2mol_1mil_GDB17_mix_pure_5.h5')

#     x_train_cat = np.load('../data/{}_x_train_cat.npy'.format(prop))
#     x_train_ani = np.load('../data/{}_x_train_ani.npy'.format(prop))
#     x_test_cat = np.load('../data/{}_x_test_cat.npy'.format(prop))
#     x_test_ani = np.load('../data/{}_x_test_ani.npy'.format(prop))
#     y_train = np.load('../data/{}_y_train.npy'.format(prop))
#     y_test = np.load('../data/{}_y_test.npy'.format(prop))
    
#     history = gen1vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     gen1vae.save('../models/gen1vae_{}_{}.h5'.format(prop,epochs))
#     with open('../models/history_gen1vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen2vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     gen2vae.save('../models/gen2vae_{}_{}.h5'.format(prop,epochs))
#     with open('../models/history_gen2vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)
        
#     history = gen3vae.autoencoder.fit([x_train_cat, x_train_ani], [x_train_cat, x_train_ani, y_train],
#                           shuffle=False,
#                           validation_data=([x_test_cat, x_test_ani], [x_test_cat, x_test_ani, y_test]),
#                           epochs=epochs)
#     gen3vae.save('../models/gen3vae_{}_{}.h5'.format(prop,epochs))
#     with open('../models/history_gen3vae_{}_{}.json'.format(prop,epochs), 'w') as f:
#         json.dump(history.history, f)

KeyboardInterrupt: 

In [11]:
epochs=1
for prop in properties:
    vae = vae_to_qspr_trainer([prop], save_as="{}_vae_{}_epochs".format(prop,epochs))

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/1
5067/5067 [==============================] - 320s 63ms/step - loss: 9.5043 - decoded_mean_loss: 6.0647 - qspr_loss: 3.4396 - decoded_mean_acc: 0.6538 - decoded_mean_mean_squared_error: 0.0136 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 3.4396 - val_loss: 4.5139 - val_decoded_mean_loss: 4.4432 - val_qspr_loss: 0.0707 - val_decoded_mean_acc: 0.6866 - val_decoded_mean_mean_squared_error: 0.0108 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.0707
Viscosity, Pa&#8226;s
total salts in training data:	3277
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 2949 samples, validate on 328 samples
Epoch 1/1
2949/2949 [==============

In [17]:
epochs=10
for prop in properties:
    vae = vae_to_qspr_trainer([prop], save_as="{}_vae_{}_epochs".format(prop,epochs), training_epochs=10)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/10
5067/5067 [==============================] - 303s 60ms/step - loss: 9.7288 - decoded_mean_loss: 6.1737 - qspr_loss: 3.5551 - decoded_mean_acc: 0.6481 - decoded_mean_mean_squared_error: 0.0139 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 3.5551 - val_loss: 4.5262 - val_decoded_mean_loss: 4.3720 - val_qspr_loss: 0.1542 - val_decoded_mean_acc: 0.6806 - val_decoded_mean_mean_squared_error: 0.0104 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.1542
Epoch 2/10
5067/5067 [==============================] - 313s 62ms/step - loss: 5.4158 - decoded_mean_loss: 3.8808 - qspr_loss: 1.5350 - decoded_mean_acc: 0.7027 - decoded_mean_mean_squared_error: 0.0095 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 1.5350 - val_loss: 

In [18]:
epochs=100
for prop in properties:
    vae = vae_to_qspr_trainer([prop], save_as="{}_vae_{}_epochs".format(prop,epochs), training_epochs=100)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/100
5067/5067 [==============================] - 314s 62ms/step - loss: 9.2228 - decoded_mean_loss: 5.5670 - qspr_loss: 3.6558 - decoded_mean_acc: 0.6588 - decoded_mean_mean_squared_error: 0.0128 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 3.6558 - val_loss: 4.7163 - val_decoded_mean_loss: 4.6283 - val_qspr_loss: 0.0879 - val_decoded_mean_acc: 0.6353 - val_decoded_mean_mean_squared_error: 0.0113 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.0879
Epoch 2/100
5067/5067 [==============================] - 304s 60ms/step - loss: 5.2883 - decoded_mean_loss: 3.6297 - qspr_loss: 1.6585 - decoded_mean_acc: 0.7117 - decoded_mean_mean_squared_error: 0.0091 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 1.6585 - val_loss

# 5M GDB Transfer Training

In [14]:
epochs=1
weights_file = "gen2_2mol_50mil_GDB17_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs))

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/1
5067/5067 [==============================] - 309s 61ms/step - loss: 9.3678 - decoded_mean_loss: 4.4969 - qspr_loss: 4.8709 - decoded_mean_acc: 0.6873 - decoded_mean_mean_squared_error: 0.0098 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 4.8709 - val_loss: 3.8008 - val_decoded_mean_loss: 3.7564 - val_qspr_loss: 0.0444 - val_decoded_mean_acc: 0.7184 - val_decoded_mean_mean_squared_error: 0.0089 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.0444
Viscosity, Pa&#8226;s
total salts in training data:	3277
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 2949 samples, validate on 328 samples
Epoch 1/1
2949/2949 [==============

In [19]:
epochs=10
weights_file = "gen2_2mol_50mil_GDB17_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs), training_epochs=epochs)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/10
5067/5067 [==============================] - 313s 62ms/step - loss: 8.8655 - decoded_mean_loss: 4.4471 - qspr_loss: 4.4183 - decoded_mean_acc: 0.6888 - decoded_mean_mean_squared_error: 0.0097 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 4.4183 - val_loss: 3.9771 - val_decoded_mean_loss: 3.8233 - val_qspr_loss: 0.1538 - val_decoded_mean_acc: 0.7168 - val_decoded_mean_mean_squared_error: 0.0091 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.1538
Epoch 2/10
5067/5067 [==============================] - 306s 60ms/step - loss: 4.7976 - decoded_mean_loss: 3.5838 - qspr_loss: 1.2137 - decoded_mean_acc: 0.7255 - decoded_mean_mean_squared_error: 0.0087 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 1.2137 - val_loss: 

In [3]:
epochs=100
weights_file = "gen2_2mol_50mil_GDB17_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs), training_epochs=epochs)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/100
5067/5067 [==============================] - 305s 60ms/step - loss: 9.4411 - decoded_mean_loss: 4.6608 - qspr_loss: 4.7803 - decoded_mean_acc: 0.6745 - decoded_mean_mean_squared_error: 0.0101 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 4.7803 - val_loss: 4.1033 - val_decoded_mean_loss: 3.9372 - val_qspr_loss: 0.1661 - val_decoded_mean_acc: 0.7002 - val_decoded_mean_mean_squared_error: 0.0093 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.1661
Epoch 2/100
5067/5067 [==============================] - 310s 61ms/step - loss: 5.0576 - decoded_mean_loss: 3.7331 - qspr_loss: 1.3246 - decoded_mean_acc: 0.7144 - decoded_mean_mean_squared_error: 0.0090 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 1.3246 - val_loss

# 5M GDB + 5M split Transfer Training

In [4]:
f = open("history_gen2_2mol_50mil_GDB17_5_2_thermal_conductivity_vae_1_epochs.json","r")
history = json.loads(f.read())

In [5]:
epochs=1
weights_file = "gen2_2mol_50mil_GDB17_5_2_mix_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs), training_epochs=epochs)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/1
5067/5067 [==============================] - 284s 56ms/step - loss: 10.2607 - decoded_mean_loss: 5.1482 - qspr_loss: 5.1125 - decoded_mean_acc: 0.7060 - decoded_mean_mean_squared_error: 0.0099 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 5.1125 - val_loss: 3.7978 - val_decoded_mean_loss: 3.5168 - val_qspr_loss: 0.2810 - val_decoded_mean_acc: 0.7567 - val_decoded_mean_mean_squared_error: 0.0079 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.2810
Viscosity, Pa&#8226;s
total salts in training data:	3277
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 2949 samples, validate on 328 samples
Epoch 1/1
2949/2949 [=============

In [6]:
epochs=10
weights_file = "gen2_2mol_50mil_GDB17_5_2_mix_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs), training_epochs=epochs)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/10
5067/5067 [==============================] - 289s 57ms/step - loss: 10.5293 - decoded_mean_loss: 5.3417 - qspr_loss: 5.1876 - decoded_mean_acc: 0.6960 - decoded_mean_mean_squared_error: 0.0102 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 5.1876 - val_loss: 3.9368 - val_decoded_mean_loss: 3.6006 - val_qspr_loss: 0.3362 - val_decoded_mean_acc: 0.7497 - val_decoded_mean_mean_squared_error: 0.0081 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.3362
Epoch 2/10
5067/5067 [==============================] - 285s 56ms/step - loss: 4.6184 - decoded_mean_loss: 3.3580 - qspr_loss: 1.2604 - decoded_mean_acc: 0.7653 - decoded_mean_mean_squared_error: 0.0077 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 1.2604 - val_loss:

In [7]:
epochs=100
weights_file = "gen2_2mol_50mil_GDB17_5_2_mix_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs), training_epochs=epochs)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/100
5067/5067 [==============================] - 298s 59ms/step - loss: 9.7809 - decoded_mean_loss: 5.1390 - qspr_loss: 4.6419 - decoded_mean_acc: 0.7047 - decoded_mean_mean_squared_error: 0.0100 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 4.6419 - val_loss: 3.7468 - val_decoded_mean_loss: 3.6441 - val_qspr_loss: 0.1027 - val_decoded_mean_acc: 0.7543 - val_decoded_mean_mean_squared_error: 0.0081 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.1027
Epoch 2/100
5067/5067 [==============================] - 292s 58ms/step - loss: 4.3079 - decoded_mean_loss: 3.2930 - qspr_loss: 1.0149 - decoded_mean_acc: 0.7713 - decoded_mean_mean_squared_error: 0.0076 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 1.0149 - val_loss

# GDB + Mix + Salt Training

In [3]:
epochs=1
weights_file = "gen2_2mol_50mil_GDB17_5_2_mix_5_2_cat_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs), training_epochs=epochs)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/1
5067/5067 [==============================] - 278s 55ms/step - loss: 9.4375 - decoded_mean_loss: 4.6341 - qspr_loss: 4.8034 - decoded_mean_acc: 0.6867 - decoded_mean_mean_squared_error: 0.0104 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 4.8034 - val_loss: 3.6650 - val_decoded_mean_loss: 3.3924 - val_qspr_loss: 0.2727 - val_decoded_mean_acc: 0.7331 - val_decoded_mean_mean_squared_error: 0.0084 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.2727
Viscosity, Pa&#8226;s
total salts in training data:	3277
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 2949 samples, validate on 328 samples
Epoch 1/1
2949/2949 [==============

In [4]:
epochs=10
weights_file = "gen2_2mol_50mil_GDB17_5_2_mix_5_2_cat_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs), training_epochs=epochs)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/10
5067/5067 [==============================] - 287s 57ms/step - loss: 9.6062 - decoded_mean_loss: 4.3772 - qspr_loss: 5.2291 - decoded_mean_acc: 0.7056 - decoded_mean_mean_squared_error: 0.0098 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 5.2291 - val_loss: 3.2710 - val_decoded_mean_loss: 3.1106 - val_qspr_loss: 0.1604 - val_decoded_mean_acc: 0.7582 - val_decoded_mean_mean_squared_error: 0.0078 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.1604
Epoch 2/10
5067/5067 [==============================] - 287s 57ms/step - loss: 4.0571 - decoded_mean_loss: 2.8829 - qspr_loss: 1.1742 - decoded_mean_acc: 0.7763 - decoded_mean_mean_squared_error: 0.0072 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 1.1742 - val_loss: 

In [5]:
epochs=100
weights_file = "gen2_2mol_50mil_GDB17_5_2_mix_5_2_cat_5_2.h5"
for prop in properties:
    vae = vae_to_qspr_trainer([prop],
                              weights_file=weights_file,
                              save_as="{}_{}_vae_{}_epochs".format(weights_file.split(".")[0],prop,epochs), training_epochs=epochs)

Specific density, kg/m<SUP>3</SUP>
total salts in training data:	5631
first training sample:	CCCC[n+]1ccc(cc1)C.[B-](F)(F)(F)F
last training sample:	C1=C[N+](CCCC)=CN1C.C(C(F)(F)F)(C(F)(F)S(=O)(=O)[O-])F
Train on 5067 samples, validate on 564 samples
Epoch 1/100
5067/5067 [==============================] - 295s 58ms/step - loss: 9.1966 - decoded_mean_loss: 4.7486 - qspr_loss: 4.4481 - decoded_mean_acc: 0.6770 - decoded_mean_mean_squared_error: 0.0107 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 4.4481 - val_loss: 3.5667 - val_decoded_mean_loss: 3.4322 - val_qspr_loss: 0.1345 - val_decoded_mean_acc: 0.7208 - val_decoded_mean_mean_squared_error: 0.0086 - val_qspr_acc: 0.0000e+00 - val_qspr_mean_squared_error: 0.1345
Epoch 2/100
5067/5067 [==============================] - 291s 57ms/step - loss: 4.3499 - decoded_mean_loss: 3.2853 - qspr_loss: 1.0646 - decoded_mean_acc: 0.7310 - decoded_mean_mean_squared_error: 0.0083 - qspr_acc: 0.0000e+00 - qspr_mean_squared_error: 1.0646 - val_loss